In [6]:
import numpy as np
from PIL import Image # Image manipulation library
%mkdir ./depth  # Just a place to store our animation frames

mkdir: ./depth: File exists


In [12]:
import json
 
# Opening JSON file
f = open('elevation.json',)
 
# returns JSON object as
# a dictionary
elevation_data = json.load(f)

In [13]:
type(elevation_data)

list

In [14]:
# Loop over a range between 0 and 100 in 0.25 increments
for i, level in enumerate(np.arange(0,100,0.25)):
    
    # First open up the satellite image we want as the background
    im = Image.open('./composite_images/satellite.png').convert('RGBA')
    
    # Create a solid image that looks like deep water (dark blue)
    overlay = Image.new('RGBA', im.size,(4,22,37,255))
    
    # Convert it into a numpy array
    ni = np.array(overlay)
    
    # Next we extract the elevation data into a numpy array
    e = np.array([np.array(xi) for xi in elevation_data])
    
    # NB: both ni and e are the same shape so we can operation onthem with numpy
    # Perform element-wise subtraction with the level
    depth = level - e


In [21]:
# Now we have the depths  we want anything > 0 to be zero alpha
alpha_mask = np.copy(depth)
alpha_mask = alpha_mask*255/alpha_mask.max()
alpha_mask  = np.where(alpha_mask<0, 0, alpha_mask)
# alpha_mask  = np.where(alpha_mask>200, 255, alpha_mask)
alpha_mask = alpha_mask**.2 
alpha_mask = alpha_mask*255/alpha_mask.max()
print(alpha_mask.min(),alpha_mask.max(),alpha_mask.mean())


247.8081618254581 255.0 251.02009040556104


In [28]:
alpha_mask

array([253.24428535, 253.24428535, 253.25032998, ..., 248.96429415,
       248.96105835, 248.96105835])

In [29]:
# Set the overlay layer opacity channel to the alpha mask
ni[...,3] = alpha_mask[...]

# Convert the numpy array back to an image
W = Image.fromarray(ni)
    
# Paste the image and save every depth frame
im.paste(W , (0,0),W)
im.save('./depth/'+ str(i).zfill(4) +'.png')

ValueError: could not broadcast input array from shape (21233664) into shape (4608,4608)